In [1]:
# Configuration de l'entraînement - IMPORTS ET SETUP
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import cv2
from PIL import Image
import json
from tqdm.auto import tqdm
import time
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Deep Learning
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import models, transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Métriques
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve
)

# Configuration plots
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 12
sns.set_style("whitegrid")

# Paths
project_root = Path.cwd().parent
sys.path.append(str(project_root))

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"📂 Projet: {project_root}")
print(f"🔥 PyTorch: {torch.__version__}")
print(f"💻 Device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Mémoire: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

e:\Master data science\MPDS3_2025\projet federal\projet\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


📂 Projet: e:\Master data science\MPDS3_2025\projet federal\projet
🔥 PyTorch: 2.9.0+cpu
💻 Device: cpu


In [7]:
# Configuration globale
CONFIG = {
    # Données
    'DATA_PATH': project_root / 'data' / 'augmented',
    'IMAGE_SIZE': (400, 400),
    'BATCH_SIZE': 16,  # Ajuster selon GPU
    'NUM_WORKERS': 4,
    
    # Classes
    'CLASSES': ['NonDemented', 'VeryMildDemented', 'MildDemented', 'ModerateDemented'],
    'NUM_CLASSES': 4,
    
    # Modèle
    'MODEL_NAME': 'resnet152',  # ou 'resnet101'
    'PRETRAINED': True,
    'FREEZE_BACKBONE': False,  # Fine-tuning complet
    
    # Entraînement
    'EPOCHS': 50,
    'LEARNING_RATE': 0.0001,
    'WEIGHT_DECAY': 1e-4,
    'MOMENTUM': 0.9,
    
    # Optimisation
    'OPTIMIZER': 'Adam',  # 'Adam' ou 'SGD'
    'SCHEDULER': 'ReduceLROnPlateau',
    'SCHEDULER_PATIENCE': 5,
    'SCHEDULER_FACTOR': 0.1,
    
    # Early Stopping
    'EARLY_STOPPING': True,
    'PATIENCE': 10,
    
    # Sauvegarde
    'SAVE_DIR': project_root / 'models',
    'LOG_DIR': project_root / 'logs',
    'FIG_DIR': project_root / 'figures',
    
    # Seed
    'SEED': 42,
}

# Créer dossiers
CONFIG['SAVE_DIR'].mkdir(exist_ok=True)
CONFIG['LOG_DIR'].mkdir(exist_ok=True)
CONFIG['FIG_DIR'].mkdir(exist_ok=True)

# Reproductibilité
torch.manual_seed(CONFIG['SEED'])
np.random.seed(CONFIG['SEED'])
if torch.cuda.is_available():
    torch.cuda.manual_seed(CONFIG['SEED'])
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

print("\n⚙️  CONFIGURATION:")
print(json.dumps({k: str(v) for k, v in CONFIG.items() if k not in ['SAVE_DIR', 'LOG_DIR', 'FIG_DIR', 'DATA_PATH']}, indent=2))


⚙️  CONFIGURATION:
{
  "IMAGE_SIZE": "(400, 400)",
  "BATCH_SIZE": "16",
  "NUM_WORKERS": "4",
  "CLASSES": "['NonDemented', 'VeryMildDemented', 'MildDemented', 'ModerateDemented']",
  "NUM_CLASSES": "4",
  "MODEL_NAME": "resnet152",
  "PRETRAINED": "True",
  "FREEZE_BACKBONE": "False",
  "EPOCHS": "50",
  "LEARNING_RATE": "0.0001",
  "WEIGHT_DECAY": "0.0001",
  "MOMENTUM": "0.9",
  "OPTIMIZER": "Adam",
  "SCHEDULER": "ReduceLROnPlateau",
  "SCHEDULER_PATIENCE": "5",
  "SCHEDULER_FACTOR": "0.1",
  "EARLY_STOPPING": "True",
  "PATIENCE": "10",
  "SEED": "42"
}


In [8]:
# Dataset et DataLoaders
class BrainMRIDataset(Dataset):
    """Dataset pour IRM cérébrales avec augmentation"""
    
    def __init__(self, root_dir, transform=None, classes=None):
        self.root_dir = Path(root_dir)
        self.transform = transform
        self.classes = classes or CONFIG['CLASSES']
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        
        # Charger images
        self.samples = []
        for class_name in self.classes:
            class_path = self.root_dir / class_name
            if class_path.exists():
                for img_path in class_path.glob('*.jpg'):
                    self.samples.append((str(img_path), self.class_to_idx[class_name]))
        
        print(f"  Loaded {len(self.samples)} samples from {root_dir.name}")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        
        # Charger image
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Appliquer transformations
        if self.transform:
            transformed = self.transform(image=image)
            image = transformed['image']
        
        return image, label

# Transformations
def get_transforms(mode='train'):
    """Pipeline d'augmentation"""
    if mode == 'train':
        return A.Compose([
            A.Resize(CONFIG['IMAGE_SIZE'][0], CONFIG['IMAGE_SIZE'][1]),
            A.HorizontalFlip(p=0.5),
            A.Rotate(limit=10, p=0.3),
            A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=10, p=0.3),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
            A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
            A.GaussianBlur(blur_limit=(3, 5), p=0.2),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])
    else:
        return A.Compose([
            A.Resize(CONFIG['IMAGE_SIZE'][0], CONFIG['IMAGE_SIZE'][1]),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])

# Créer datasets
print("\n📦 CRÉATION DES DATASETS")
print("=" * 60)

train_dataset = BrainMRIDataset(
    root_dir=CONFIG['DATA_PATH'] / 'train',
    transform=get_transforms('train'),
    classes=CONFIG['CLASSES']
)

val_dataset = BrainMRIDataset(
    root_dir=CONFIG['DATA_PATH'] / 'val',
    transform=get_transforms('val'),
    classes=CONFIG['CLASSES']
)

test_dataset = BrainMRIDataset(
    root_dir=CONFIG['DATA_PATH'] / 'test',
    transform=get_transforms('test'),
    classes=CONFIG['CLASSES']
)

# DataLoaders
train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG['BATCH_SIZE'],
    shuffle=True,
    num_workers=CONFIG['NUM_WORKERS'],
    pin_memory=True if torch.cuda.is_available() else False
)

val_loader = DataLoader(
    val_dataset,
    batch_size=CONFIG['BATCH_SIZE'],
    shuffle=False,
    num_workers=CONFIG['NUM_WORKERS'],
    pin_memory=True if torch.cuda.is_available() else False
)

test_loader = DataLoader(
    test_dataset,
    batch_size=CONFIG['BATCH_SIZE'],
    shuffle=False,
    num_workers=CONFIG['NUM_WORKERS'],
    pin_memory=True if torch.cuda.is_available() else False
)

print(f"\n✅ Train: {len(train_dataset)} images, {len(train_loader)} batches")
print(f"✅ Val:   {len(val_dataset)} images, {len(val_loader)} batches")
print(f"✅ Test:  {len(test_dataset)} images, {len(test_loader)} batches")


📦 CRÉATION DES DATASETS
  Loaded 32391 samples from train
  Loaded 1994 samples from val
  Loaded 3086 samples from test

✅ Train: 32391 images, 2025 batches
✅ Val:   1994 images, 125 batches
✅ Test:  3086 images, 193 batches


In [9]:
# Création du modèle
def create_model(model_name='resnet152', num_classes=4, pretrained=True, freeze_backbone=False):
    """Crée et configure le modèle ResNet"""
    
    print(f"\n🏗️  CRÉATION DU MODÈLE: {model_name.upper()}")
    print("=" * 60)
    
    # Charger modèle pretrained
    if model_name == 'resnet152':
        if pretrained:
            weights = models.ResNet152_Weights.IMAGENET1K_V2
            model = models.resnet152(weights=weights)
        else:
            model = models.resnet152(weights=None)
    elif model_name == 'resnet101':
        if pretrained:
            weights = models.ResNet101_Weights.IMAGENET1K_V2
            model = models.resnet101(weights=weights)
        else:
            model = models.resnet101(weights=None)
    else:
        raise ValueError(f"Modèle {model_name} non supporté")
    
    print(f"✅ Modèle chargé: {model_name}")
    print(f"   Pretrained: {pretrained}")
    
    # Geler le backbone si demandé
    if freeze_backbone:
        for param in model.parameters():
            param.requires_grad = False
        print(f"❄️  Backbone gelé (fine-tuning tête seulement)")
    else:
        print(f"🔥 Backbone dégelé (fine-tuning complet)")
    
    # Modifier la dernière couche
    num_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(num_features, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, num_classes)
    )
    
    print(f"\n📊 Architecture de la tête:")
    print(f"   Input: {num_features} features")
    print(f"   Hidden: 512 neurons (Dropout 0.5, ReLU, Dropout 0.3)")
    print(f"   Output: {num_classes} classes")
    
    # Compter paramètres
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"\n📈 Paramètres:")
    print(f"   Total: {total_params:,}")
    print(f"   Entraînables: {trainable_params:,}")
    print(f"   Gelés: {total_params - trainable_params:,}")
    
    return model

# Créer modèle
model = create_model(
    model_name=CONFIG['MODEL_NAME'],
    num_classes=CONFIG['NUM_CLASSES'],
    pretrained=CONFIG['PRETRAINED'],
    freeze_backbone=CONFIG['FREEZE_BACKBONE']
)

# Déplacer sur GPU
model = model.to(device)
print(f"\n✅ Modèle déplacé sur: {device}")


🏗️  CRÉATION DU MODÈLE: RESNET152
✅ Modèle chargé: resnet152
   Pretrained: True
🔥 Backbone dégelé (fine-tuning complet)

📊 Architecture de la tête:
   Input: 2048 features
   Hidden: 512 neurons (Dropout 0.5, ReLU, Dropout 0.3)
   Output: 4 classes

📈 Paramètres:
   Total: 59,194,948
   Entraînables: 59,194,948
   Gelés: 0

✅ Modèle déplacé sur: cpu


In [10]:
# Configuration de l'entraînement - CORRIGÉE
# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
if CONFIG['OPTIMIZER'] == 'Adam':
    optimizer = optim.Adam(
        model.parameters(),
        lr=CONFIG['LEARNING_RATE'],
        weight_decay=CONFIG['WEIGHT_DECAY']
    )
elif CONFIG['OPTIMIZER'] == 'SGD':
    optimizer = optim.SGD(
        model.parameters(),
        lr=CONFIG['LEARNING_RATE'],
        momentum=CONFIG['MOMENTUM'],
        weight_decay=CONFIG['WEIGHT_DECAY']
    )

# Learning rate scheduler - CORRECTION: suppression du paramètre 'verbose'
if CONFIG['SCHEDULER'] == 'ReduceLROnPlateau':
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=CONFIG['SCHEDULER_FACTOR'],
        patience=CONFIG['SCHEDULER_PATIENCE']
    )
elif CONFIG['SCHEDULER'] == 'CosineAnnealingLR':
    scheduler = optim.lr_scheduler.CosineAnnealingLR(
        optimizer,
        T_max=CONFIG['EPOCHS'],
        eta_min=1e-6
    )

print("\n⚙️  CONFIGURATION D'ENTRAÎNEMENT")
print("=" * 60)
print(f"Loss: CrossEntropyLoss")
print(f"Optimizer: {CONFIG['OPTIMIZER']}")
print(f"Learning Rate: {CONFIG['LEARNING_RATE']}")
print(f"Weight Decay: {CONFIG['WEIGHT_DECAY']}")
print(f"Scheduler: {CONFIG['SCHEDULER']}")
print(f"Early Stopping: {CONFIG['EARLY_STOPPING']} (patience={CONFIG['PATIENCE']})")


⚙️  CONFIGURATION D'ENTRAÎNEMENT
Loss: CrossEntropyLoss
Optimizer: Adam
Learning Rate: 0.0001
Weight Decay: 0.0001
Scheduler: ReduceLROnPlateau
Early Stopping: True (patience=10)


In [11]:
# Fonctions d'entraînement
def train_epoch(model, loader, criterion, optimizer, device):
    """Entraîne une epoch"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(loader, desc='Training')
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        
        # Forward
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward
        loss.backward()
        optimizer.step()
        
        # Statistiques
        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        # Mise à jour barre
        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{100.*correct/total:.2f}%'
        })
    
    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    
    return epoch_loss, epoch_acc

def validate_epoch(model, loader, criterion, device):
    """Valide une epoch"""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        pbar = tqdm(loader, desc='Validation')
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            
            # Forward
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Statistiques
            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            # Sauvegarder pour métriques
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(torch.softmax(outputs, dim=1).cpu().numpy())
            
            pbar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100.*correct/total:.2f}%'
            })
    
    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    
    return epoch_loss, epoch_acc, all_preds, all_labels, all_probs

print("✅ Fonctions d'entraînement prêtes")

✅ Fonctions d'entraînement prêtes


In [ ]:
# BOUCLE D'ENTRAÎNEMENT PRINCIPALE
print("\n🚀 DÉBUT DE L'ENTRAÎNEMENT")
print("=" * 60)
print(f"Epochs: {CONFIG['EPOCHS']}")
print(f"Batch size: {CONFIG['BATCH_SIZE']}")
print(f"Device: {device}")
print("=" * 60)

# Historique
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': [],
    'lr': []
}

# Early stopping
best_val_loss = float('inf')
patience_counter = 0
best_model_path = CONFIG['SAVE_DIR'] / f"{CONFIG['MODEL_NAME']}_best.pth"

start_time = time.time()

for epoch in range(CONFIG['EPOCHS']):
    print(f"\n📅 Epoch {epoch+1}/{CONFIG['EPOCHS']}")
    print("-" * 60)
    
    # Entraînement
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # Validation
    val_loss, val_acc, val_preds, val_labels, val_probs = validate_epoch(
        model, val_loader, criterion, device
    )
    
    # Scheduler
    if CONFIG['SCHEDULER'] == 'ReduceLROnPlateau':
        scheduler.step(val_loss)
    else:
        scheduler.step()
    
    current_lr = optimizer.param_groups[0]['lr']
    
    # Sauvegarder historique
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['lr'].append(current_lr)
    
    # Affichage des résultats de l'epoch
    print(f"\n📊 Résultats Epoch {epoch+1}:")
    print(f"   Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"   Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.2f}%")
    print(f"   Learning Rate: {current_lr:.6f}")
    
    # Sauvegarde du meilleur modèle
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        
        # Sauvegarder le modèle
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'val_loss': val_loss,
            'val_acc': val_acc,
            'history': history,
            'config': CONFIG
        }, best_model_path)
        
        print(f"💾 Meilleur modèle sauvegardé! (val_loss: {val_loss:.4f})")
    else:
        patience_counter += 1
        print(f"⏳ Early stopping: {patience_counter}/{CONFIG['PATIENCE']}")
    
    # Early stopping
    if CONFIG['EARLY_STOPPING'] and patience_counter >= CONFIG['PATIENCE']:
        print(f"\n🛑 ARRÊT PRÉCOCE ACTIVÉ après {epoch+1} epochs!")
        print(f"   Meilleur val_loss: {best_val_loss:.4f}")
        break

# Calcul du temps total
end_time = time.time()
total_time = end_time - start_time

print(f"\n✅ ENTRAÎNEMENT TERMINÉ en {total_time/60:.2f} minutes")
print(f"   Meilleur val_loss: {best_val_loss:.4f}")

# Sauvegarde de l'historique complet
history_file = CONFIG['LOG_DIR'] / f"{CONFIG['MODEL_NAME']}_history.json"
with open(history_file, 'w') as f:
    json.dump(history, f, indent=2)

print(f"💾 Historique sauvegardé: {history_file}")


🚀 DÉBUT DE L'ENTRAÎNEMENT
Epochs: 50
Batch size: 16
Device: cpu

📅 Epoch 1/50
------------------------------------------------------------


Training:   0%|          | 0/2025 [00:00<?, ?it/s]

In [14]:
# CELLULE 1: DÉFINITION DE LA CLASSE DATASET
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import cv2
from PIL import Image
import json
from tqdm.auto import tqdm
import time
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Deep Learning
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import models, transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.cuda.amp import autocast, GradScaler

# Métriques
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve
)

print("✅ Tous les imports sont faits!")

class BrainMRIDataset(Dataset):
    """Dataset pour IRM cérébrales avec augmentation"""
    
    def __init__(self, root_dir, transform=None, classes=None):
        self.root_dir = Path(root_dir)
        self.transform = transform
        self.classes = classes or ['NonDemented', 'VeryMildDemented', 'MildDemented', 'ModerateDemented']
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        
        # Charger images
        self.samples = []
        for class_name in self.classes:
            class_path = self.root_dir / class_name
            if class_path.exists():
                for img_path in class_path.glob('*.jpg'):
                    self.samples.append((str(img_path), self.class_to_idx[class_name]))
            else:
                print(f"⚠️  Dossier non trouvé: {class_path}")
        
        print(f"  ✅ Chargé {len(self.samples)} échantillons depuis {root_dir.name}")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        
        # Charger image
        image = cv2.imread(img_path)
        if image is None:
            print(f"❌ Impossible de charger: {img_path}")
            # Retourner une image noire en cas d'erreur
            image = np.zeros((224, 224, 3), dtype=np.uint8)
        else:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Appliquer transformations
        if self.transform:
            transformed = self.transform(image=image)
            image = transformed['image']
        
        return image, label

print("✅ Classe BrainMRIDataset définie!")

✅ Tous les imports sont faits!
✅ Classe BrainMRIDataset définie!


In [15]:
# CELLULE 2: CONFIGURATION
# Paths
project_root = Path.cwd().parent
sys.path.append(str(project_root))

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"💻 Device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Mémoire: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

CONFIG = {
    'DATA_PATH': project_root / 'data' / 'augmented',
    'IMAGE_SIZE': (256, 256),
    'BATCH_SIZE': 32,
    'NUM_WORKERS': 6,
    'CLASSES': ['NonDemented', 'VeryMildDemented', 'MildDemented', 'ModerateDemented'],
    'NUM_CLASSES': 4,
    'MODEL_NAME': 'resnet101',
    'PRETRAINED': True,
    'FREEZE_BACKBONE': False,
    'EPOCHS': 40,
    'LEARNING_RATE': 0.0005,
    'WEIGHT_DECAY': 1e-4,
    'OPTIMIZER': 'AdamW',
    'SCHEDULER': 'CosineAnnealingLR',
    'EARLY_STOPPING': True,
    'PATIENCE': 8,
    'SAVE_DIR': project_root / 'models',
    'LOG_DIR': project_root / 'logs', 
    'FIG_DIR': project_root / 'figures',
    'SEED': 42,
}

# Créer dossiers
CONFIG['SAVE_DIR'].mkdir(exist_ok=True)
CONFIG['LOG_DIR'].mkdir(exist_ok=True)
CONFIG['FIG_DIR'].mkdir(exist_ok=True)

print("✅ Configuration définie!")

💻 Device: cpu
✅ Configuration définie!


In [6]:
# CELLULE 1: FONCTION DE CRÉATION DU MODÈLE
def create_model(model_name='resnet101', num_classes=4, pretrained=True, freeze_backbone=False):
    """Crée et configure le modèle ResNet"""
    
    print(f"🏗️  CRÉATION DU MODÈLE: {model_name.upper()}")
    print("=" * 60)
    
    # Charger modèle pretrained
    if model_name == 'resnet152':
        if pretrained:
            weights = models.ResNet152_Weights.IMAGENET1K_V2
            model = models.resnet152(weights=weights)
        else:
            model = models.resnet152(weights=None)
    elif model_name == 'resnet101':
        if pretrained:
            weights = models.ResNet101_Weights.IMAGENET1K_V2
            model = models.resnet101(weights=weights)
        else:
            model = models.resnet101(weights=None)
    elif model_name == 'resnet50':
        if pretrained:
            weights = models.ResNet50_Weights.IMAGENET1K_V2
            model = models.resnet50(weights=weights)
        else:
            model = models.resnet50(weights=None)
    else:
        raise ValueError(f"Modèle {model_name} non supporté")
    
    print(f"✅ Modèle chargé: {model_name}")
    print(f"   Pretrained: {pretrained}")
    
    # Geler le backbone si demandé
    if freeze_backbone:
        for param in model.parameters():
            param.requires_grad = False
        print(f"❄️  Backbone gelé (fine-tuning tête seulement)")
    else:
        print(f"🔥 Backbone dégelé (fine-tuning complet)")
    
    # Modifier la dernière couche
    num_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(num_features, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, num_classes)
    )
    
    print(f"📊 Architecture de la tête:")
    print(f"   Input: {num_features} features")
    print(f"   Hidden: 512 neurons (Dropout 0.5, ReLU, Dropout 0.3)")
    print(f"   Output: {num_classes} classes")
    
    # Compter paramètres
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"📈 Paramètres:")
    print(f"   Total: {total_params:,}")
    print(f"   Entraînables: {trainable_params:,}")
    print(f"   Gelés: {total_params - trainable_params:,}")
    
    return model

print("✅ Fonction create_model définie!")

✅ Fonction create_model définie!


In [7]:
# 1. CONFIGURATION OPTIMISÉE
CONFIG = {
    'DATA_PATH': project_root / 'data' / 'augmented',
    'IMAGE_SIZE': (256, 256),
    'BATCH_SIZE': 32,
    'NUM_WORKERS': 6,
    'CLASSES': ['NonDemented', 'VeryMildDemented', 'MildDemented', 'ModerateDemented'],
    'NUM_CLASSES': 4,
    'MODEL_NAME': 'resnet101',
    'PRETRAINED': True,
    'FREEZE_BACKBONE': False,
    'EPOCHS': 40,
    'LEARNING_RATE': 0.0005,
    'WEIGHT_DECAY': 1e-4,
    'OPTIMIZER': 'AdamW',
    'SCHEDULER': 'CosineAnnealingLR',
    'EARLY_STOPPING': True,
    'PATIENCE': 8,
    'SAVE_DIR': project_root / 'models',
    'LOG_DIR': project_root / 'logs', 
    'FIG_DIR': project_root / 'figures',
    'SEED': 42,
}

# 2. CRÉATION DU MODÈLE D'ABORD
print("🏗️  CRÉATION DU MODÈLE...")
model = create_model(
    model_name=CONFIG['MODEL_NAME'],
    num_classes=CONFIG['NUM_CLASSES'],
    pretrained=CONFIG['PRETRAINED'],
    freeze_backbone=CONFIG['FREEZE_BACKBONE']
)
model = model.to(device)

# 3. CONFIGURATION OPTIMIZER APRÈS
print("⚙️  CONFIGURATION DE L'OPTIMIZER...")

# Mixed Precision Scaler
scaler = GradScaler() if torch.cuda.is_available() else None

# Optimizer
if CONFIG['OPTIMIZER'] == 'AdamW':
    optimizer = optim.AdamW(
        model.parameters(),
        lr=CONFIG['LEARNING_RATE'],
        weight_decay=CONFIG['WEIGHT_DECAY'],
        betas=(0.9, 0.999)
    )
else:
    optimizer = optim.Adam(
        model.parameters(),
        lr=CONFIG['LEARNING_RATE'],
        weight_decay=CONFIG['WEIGHT_DECAY']
    )

# Scheduler
scheduler = optim.lr_scheduler.CosineAnnealingLR(
    optimizer, 
    T_max=CONFIG['EPOCHS'],
    eta_min=1e-6
)

criterion = nn.CrossEntropyLoss()

print("✅ Configuration terminée!")

🏗️  CRÉATION DU MODÈLE...
🏗️  CRÉATION DU MODÈLE: RESNET101
Downloading: "https://download.pytorch.org/models/resnet101-cd907fc2.pth" to C:\Users\adnan/.cache\torch\hub\checkpoints\resnet101-cd907fc2.pth


100%|██████████| 171M/171M [02:01<00:00, 1.47MB/s] 


✅ Modèle chargé: resnet101
   Pretrained: True
🔥 Backbone dégelé (fine-tuning complet)
📊 Architecture de la tête:
   Input: 2048 features
   Hidden: 512 neurons (Dropout 0.5, ReLU, Dropout 0.3)
   Output: 4 classes
📈 Paramètres:
   Total: 43,551,300
   Entraînables: 43,551,300
   Gelés: 0
⚙️  CONFIGURATION DE L'OPTIMIZER...
✅ Configuration terminée!


In [16]:
# CELLULE 4: CRÉATION DES DATALOADERS
print("📦 CRÉATION DES DATASETS ET DATALOADERS...")

# Transformations optimisées
def get_transforms_optimized(mode='train'):
    """Pipeline d'augmentation optimisé"""
    if mode == 'train':
        return A.Compose([
            A.Resize(CONFIG['IMAGE_SIZE'][0], CONFIG['IMAGE_SIZE'][1]),
            A.HorizontalFlip(p=0.5),
            A.Rotate(limit=10, p=0.3),
            A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=10, p=0.3),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
            A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])
    else:
        return A.Compose([
            A.Resize(CONFIG['IMAGE_SIZE'][0], CONFIG['IMAGE_SIZE'][1]),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])

# Créer datasets
train_dataset = BrainMRIDataset(
    root_dir=CONFIG['DATA_PATH'] / 'train',
    transform=get_transforms_optimized('train'),
    classes=CONFIG['CLASSES']
)

val_dataset = BrainMRIDataset(
    root_dir=CONFIG['DATA_PATH'] / 'val',
    transform=get_transforms_optimized('val'),
    classes=CONFIG['CLASSES']
)

test_dataset = BrainMRIDataset(
    root_dir=CONFIG['DATA_PATH'] / 'test',
    transform=get_transforms_optimized('test'),
    classes=CONFIG['CLASSES']
)

# DataLoaders optimisés
train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG['BATCH_SIZE'],
    shuffle=True,
    num_workers=CONFIG['NUM_WORKERS'],
    pin_memory=True,
    persistent_workers=True if CONFIG['NUM_WORKERS'] > 0 else False,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=CONFIG['BATCH_SIZE'],
    shuffle=False,
    num_workers=CONFIG['NUM_WORKERS'],
    pin_memory=True,
    persistent_workers=True if CONFIG['NUM_WORKERS'] > 0 else False,
)

test_loader = DataLoader(
    test_dataset,
    batch_size=CONFIG['BATCH_SIZE'],
    shuffle=False,
    num_workers=CONFIG['NUM_WORKERS'],
    pin_memory=True,
    persistent_workers=True if CONFIG['NUM_WORKERS'] > 0 else False,
)

print(f"✅ Train: {len(train_dataset)} images, {len(train_loader)} batches")
print(f"✅ Val:   {len(val_dataset)} images, {len(val_loader)} batches")
print(f"✅ Test:  {len(test_dataset)} images, {len(test_loader)} batches")

📦 CRÉATION DES DATASETS ET DATALOADERS...
  ✅ Chargé 32391 échantillons depuis train
  ✅ Chargé 1994 échantillons depuis val
  ✅ Chargé 3086 échantillons depuis test
✅ Train: 32391 images, 1013 batches
✅ Val:   1994 images, 63 batches
✅ Test:  3086 images, 97 batches


In [17]:
# CELLULE 5: CRÉATION DU MODÈLE ET OPTIMIZER
print("🏗️  CRÉATION DU MODÈLE...")
model = create_model(
    model_name=CONFIG['MODEL_NAME'],
    num_classes=CONFIG['NUM_CLASSES'],
    pretrained=CONFIG['PRETRAINED'],
    freeze_backbone=CONFIG['FREEZE_BACKBONE']
)
model = model.to(device)
print(f"✅ Modèle créé et déplacé sur: {device}")

print("⚙️  CONFIGURATION DE L'OPTIMIZER...")

# Mixed Precision Scaler
scaler = GradScaler() if torch.cuda.is_available() else None

# Optimizer
if CONFIG['OPTIMIZER'] == 'AdamW':
    optimizer = optim.AdamW(
        model.parameters(),
        lr=CONFIG['LEARNING_RATE'],
        weight_decay=CONFIG['WEIGHT_DECAY'],
        betas=(0.9, 0.999)
    )
    print("✅ Optimizer: AdamW")
else:
    optimizer = optim.Adam(
        model.parameters(),
        lr=CONFIG['LEARNING_RATE'],
        weight_decay=CONFIG['WEIGHT_DECAY']
    )
    print("✅ Optimizer: Adam")

# Scheduler
scheduler = optim.lr_scheduler.CosineAnnealingLR(
    optimizer, 
    T_max=CONFIG['EPOCHS'],
    eta_min=1e-6
)

criterion = nn.CrossEntropyLoss()

print("✅ Configuration training terminée!")

🏗️  CRÉATION DU MODÈLE...
🏗️  CRÉATION DU MODÈLE: RESNET101
✅ Modèle chargé: resnet101
   Pretrained: True
🔥 Backbone dégelé (fine-tuning complet)
📊 Architecture de la tête:
   Input: 2048 features
   Hidden: 512 neurons (Dropout 0.5, ReLU, Dropout 0.3)
   Output: 4 classes
📈 Paramètres:
   Total: 43,551,300
   Entraînables: 43,551,300
   Gelés: 0
✅ Modèle créé et déplacé sur: cpu
⚙️  CONFIGURATION DE L'OPTIMIZER...
✅ Optimizer: AdamW
✅ Configuration training terminée!


In [18]:
# CELLULE 6: FONCTIONS D'ENTRAÎNEMENT
def train_epoch_fast(model, loader, criterion, optimizer, device, scaler):
    """Entraînement avec Mixed Precision"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(loader, desc='Training')
    for images, labels in pbar:
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        
        # Forward avec Mixed Precision
        with autocast(enabled=scaler is not None):
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        # Backward avec gradient scaling
        optimizer.zero_grad()
        if scaler:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()
        
        # Statistiques
        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        # Mise à jour barre
        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{100.*correct/total:.2f}%'
        })
    
    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    
    return epoch_loss, epoch_acc

def validate_epoch_fast(model, loader, criterion, device):
    """Validation optimisée"""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        pbar = tqdm(loader, desc='Validation')
        for images, labels in pbar:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            
            # Forward avec autocast pour cohérence
            with autocast(enabled=torch.cuda.is_available()):
                outputs = model(images)
                loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
            pbar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100.*correct/total:.2f}%'
            })
    
    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    
    return epoch_loss, epoch_acc, all_preds, all_labels

def print_gpu_usage():
    """Affiche l'utilisation du GPU"""
    if torch.cuda.is_available():
        print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
        print(f"💾 Mémoire utilisée: {torch.cuda.memory_allocated(0)/1024**3:.2f} GB")
        print(f"💾 Mémoire réservée: {torch.cuda.memory_reserved(0)/1024**3:.2f} GB")

print("✅ Fonctions d'entraînement définies!")

✅ Fonctions d'entraînement définies!


In [ ]:
# CELLULE 7: BOUCLE D'ENTRAÎNEMENT PRINCIPALE
print("\n🚀 DÉBUT DE L'ENTRAÎNEMENT OPTIMISÉ")
print("=" * 60)
print(f"Modèle: {CONFIG['MODEL_NAME']}")
print(f"Batch size: {CONFIG['BATCH_SIZE']}")
print(f"Image size: {CONFIG['IMAGE_SIZE']}")
print(f"Mixed Precision: {'ACTIVÉ' if torch.cuda.is_available() else 'DÉSACTIVÉ (CPU)'}")
print(f"Device: {device}")
print(f"Train samples: {len(train_dataset)}")
print(f"Val samples: {len(val_dataset)}")
print("=" * 60)

# Historique
history = {
    'train_loss': [], 'train_acc': [],
    'val_loss': [], 'val_acc': [],
    'lr': []
}

# Early stopping
best_val_acc = 0.0
patience_counter = 0
best_model_path = CONFIG['SAVE_DIR'] / f"{CONFIG['MODEL_NAME']}_best.pth"

start_time = time.time()

for epoch in range(CONFIG['EPOCHS']):
    print(f"\n📅 Epoch {epoch+1}/{CONFIG['EPOCHS']}")
    print("-" * 50)
    
    # Entraînement accéléré
    train_loss, train_acc = train_epoch_fast(
        model, train_loader, criterion, optimizer, device, scaler
    )
    
    # Validation
    val_loss, val_acc, val_preds, val_labels = validate_epoch_fast(
        model, val_loader, criterion, device
    )
    
    # Scheduler
    scheduler.step()
    current_lr = optimizer.param_groups[0]['lr']
    
    # Sauvegarder historique
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['lr'].append(current_lr)
    
    # Affichage
    print(f"📊 Train: {train_acc:.2f}% | Val: {val_acc:.2f}% | LR: {current_lr:.6f}")
    
    # Sauvegarde du meilleur modèle (basé sur accuracy)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'val_acc': val_acc,
            'val_loss': val_loss,
            'history': history,
            'config': CONFIG
        }, best_model_path)
        
        print(f"💾 Meilleur modèle sauvegardé! (val_acc: {val_acc:.2f}%)")
    else:
        patience_counter += 1
        print(f"⏳ Early stopping: {patience_counter}/{CONFIG['PATIENCE']}")
    
    # Early stopping
    if CONFIG['EARLY_STOPPING'] and patience_counter >= CONFIG['PATIENCE']:
        print(f"\n🛑 ARRÊT PRÉCOCE ACTIVÉ après {epoch+1} epochs!")
        print(f"   Meilleur val_acc: {best_val_acc:.2f}%")
        break

# Temps total
end_time = time.time()
total_time = (end_time - start_time) / 60

print(f"\n✅ ENTRAÎNEMENT TERMINÉ en {total_time:.2f} minutes")
print(f"🎯 Meilleure val_acc: {best_val_acc:.2f}%")

# Sauvegarde de l'historique
history_file = CONFIG['LOG_DIR'] / f"{CONFIG['MODEL_NAME']}_history.json"
with open(history_file, 'w') as f:
    # Convertir les numpy arrays en listes pour JSON
    history_serializable = {
        'train_loss': [float(x) for x in history['train_loss']],
        'train_acc': [float(x) for x in history['train_acc']],
        'val_loss': [float(x) for x in history['val_loss']],
        'val_acc': [float(x) for x in history['val_acc']],
        'lr': [float(x) for x in history['lr']]
    }
    json.dump(history_serializable, f, indent=2)s
print(f"💾 Historique sauvegardé: {history_file}")


🚀 DÉBUT DE L'ENTRAÎNEMENT OPTIMISÉ
Modèle: resnet101
Batch size: 32
Image size: (256, 256)
Mixed Precision: DÉSACTIVÉ (CPU)
Device: cpu
Train samples: 32391
Val samples: 1994

📅 Epoch 1/40
--------------------------------------------------


Training:   0%|          | 0/1013 [00:00<?, ?it/s]

In [8]:
# AJOUTER ces imports après les autres
from torch.cuda.amp import autocast, GradScaler

In [9]:
# ⚡ CONFIGURATION AVEC MIXED PRECISION
print("🔥 Activation du Mixed Precision pour GPU NVIDIA...")

# Mixed Precision Scaler - ESSENTIEL pour la performance
scaler = GradScaler() if torch.cuda.is_available() else None

# Optimizer amélioré
if CONFIG['OPTIMIZER'] == 'AdamW':
    optimizer = optim.AdamW(
        model.parameters(),
        lr=CONFIG['LEARNING_RATE'],
        weight_decay=CONFIG['WEIGHT_DECAY'],
        betas=(0.9, 0.999)
    )
else:
    optimizer = optim.Adam(
        model.parameters(),
        lr=CONFIG['LEARNING_RATE'],
        weight_decay=CONFIG['WEIGHT_DECAY']
    )

# Scheduler Cosine - Excellent convergence
scheduler = optim.lr_scheduler.CosineAnnealingLR(
    optimizer, 
    T_max=CONFIG['EPOCHS'],
    eta_min=1e-6
)

criterion = nn.CrossEntropyLoss()

🔥 Activation du Mixed Precision pour GPU NVIDIA...


In [11]:
def train_epoch_fast(model, loader, criterion, optimizer, device, scaler):
    """Entraînement avec Mixed Precision"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(loader, desc='Training')
    for images, labels in pbar:
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        
        # Forward avec Mixed Precision
        with autocast(enabled=scaler is not None):
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        # Backward avec gradient scaling
        optimizer.zero_grad()
        if scaler:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()
        
        # Statistiques
        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        # Mise à jour barre
        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{100.*correct/total:.2f}%'
        })
    
    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    
    return epoch_loss, epoch_acc

def validate_epoch_fast(model, loader, criterion, device):
    """Validation optimisée"""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        pbar = tqdm(loader, desc='Validation')
        for images, labels in pbar:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            
            # Forward avec autocast pour cohérence
            with autocast(enabled=torch.cuda.is_available()):
                outputs = model(images)
                loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
            pbar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100.*correct/total:.2f}%'
            })
    
    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    
    return epoch_loss, epoch_acc, all_preds, all_labels

In [13]:
# CELLULE 1: CRÉATION DES DATASETS ET DATALOADERS
print("📦 CRÉATION DES DATASETS ET DATALOADERS...")

# Transformations optimisées
def get_transforms_optimized(mode='train'):
    """Pipeline d'augmentation optimisé"""
    if mode == 'train':
        return A.Compose([
            A.Resize(CONFIG['IMAGE_SIZE'][0], CONFIG['IMAGE_SIZE'][1]),
            A.HorizontalFlip(p=0.5),
            A.Rotate(limit=10, p=0.3),
            A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=10, p=0.3),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
            A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])
    else:
        return A.Compose([
            A.Resize(CONFIG['IMAGE_SIZE'][0], CONFIG['IMAGE_SIZE'][1]),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])

# Créer datasets
train_dataset = BrainMRIDataset(
    root_dir=CONFIG['DATA_PATH'] / 'train',
    transform=get_transforms_optimized('train'),
    classes=CONFIG['CLASSES']
)

val_dataset = BrainMRIDataset(
    root_dir=CONFIG['DATA_PATH'] / 'val',
    transform=get_transforms_optimized('val'),
    classes=CONFIG['CLASSES']
)

test_dataset = BrainMRIDataset(
    root_dir=CONFIG['DATA_PATH'] / 'test',
    transform=get_transforms_optimized('test'),
    classes=CONFIG['CLASSES']
)

# DataLoaders optimisés
train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG['BATCH_SIZE'],
    shuffle=True,
    num_workers=CONFIG['NUM_WORKERS'],
    pin_memory=True,
    persistent_workers=True if CONFIG['NUM_WORKERS'] > 0 else False,
    prefetch_factor=2 if CONFIG['NUM_WORKERS'] > 0 else None,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=CONFIG['BATCH_SIZE'],
    shuffle=False,
    num_workers=CONFIG['NUM_WORKERS'],
    pin_memory=True,
    persistent_workers=True if CONFIG['NUM_WORKERS'] > 0 else False,
)

test_loader = DataLoader(
    test_dataset,
    batch_size=CONFIG['BATCH_SIZE'],
    shuffle=False,
    num_workers=CONFIG['NUM_WORKERS'],
    pin_memory=True,
    persistent_workers=True if CONFIG['NUM_WORKERS'] > 0 else False,
)

print(f"✅ Train: {len(train_dataset)} images, {len(train_loader)} batches")
print(f"✅ Val:   {len(val_dataset)} images, {len(val_loader)} batches")
print(f"✅ Test:  {len(test_dataset)} images, {len(test_loader)} batches")

📦 CRÉATION DES DATASETS ET DATALOADERS...


NameError: name 'BrainMRIDataset' is not defined

In [12]:
# 🚀 BOUCLE D'ENTRAÎNEMENT HAUTE PERFORMANCE
print("🚀 DÉBUT DE L'ENTRAÎNEMENT OPTIMISÉ SUR GPU NVIDIA")
print("=" * 60)
print(f"Modèle: {CONFIG['MODEL_NAME']}")
print(f"Batch size: {CONFIG['BATCH_SIZE']}")
print(f"Image size: {CONFIG['IMAGE_SIZE']}")
print(f"Mixed Precision: ACTIVÉ")
print(f"Device: {device}")
print("=" * 60)

# Historique
history = {
    'train_loss': [], 'train_acc': [],
    'val_loss': [], 'val_acc': [],
    'lr': []
}

# Early stopping
best_val_acc = 0.0
patience_counter = 0
best_model_path = CONFIG['SAVE_DIR'] / f"{CONFIG['MODEL_NAME']}_best.pth"

start_time = time.time()

for epoch in range(CONFIG['EPOCHS']):
    print(f"\n📅 Epoch {epoch+1}/{CONFIG['EPOCHS']}")
    print("-" * 50)
    
    # Entraînement accéléré
    train_loss, train_acc = train_epoch_fast(
        model, train_loader, criterion, optimizer, device, scaler
    )
    
    # Validation
    val_loss, val_acc, val_preds, val_labels = validate_epoch_fast(
        model, val_loader, criterion, device
    )
    
    # Scheduler
    scheduler.step()
    current_lr = optimizer.param_groups[0]['lr']
    
    # Sauvegarder historique
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['lr'].append(current_lr)
    
    # Affichage
    print(f"📊 Train: {train_acc:.2f}% | Val: {val_acc:.2f}% | LR: {current_lr:.6f}")
    
    # Sauvegarde du meilleur modèle (basé sur accuracy)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'val_acc': val_acc,
            'val_loss': val_loss,
            'history': history,
            'config': CONFIG
        }, best_model_path)
        
        print(f"💾 Meilleur modèle sauvegardé! (val_acc: {val_acc:.2f}%)")
    else:
        patience_counter += 1
        print(f"⏳ Early stopping: {patience_counter}/{CONFIG['PATIENCE']}")
    
    # Early stopping
    if CONFIG['EARLY_STOPPING'] and patience_counter >= CONFIG['PATIENCE']:
        print(f"\n🛑 ARRÊT PRÉCOCE ACTIVÉ après {epoch+1} epochs!")
        print(f"   Meilleur val_acc: {best_val_acc:.2f}%")
        break

# Temps total
end_time = time.time()
total_time = (end_time - start_time) / 60

print(f"\n✅ ENTRAÎNEMENT TERMINÉ en {total_time:.2f} minutes")
print(f"🎯 Meilleure val_acc: {best_val_acc:.2f}%")

🚀 DÉBUT DE L'ENTRAÎNEMENT OPTIMISÉ SUR GPU NVIDIA
Modèle: resnet101
Batch size: 32
Image size: (256, 256)
Mixed Precision: ACTIVÉ
Device: cpu

📅 Epoch 1/40
--------------------------------------------------


NameError: name 'train_loader' is not defined

In [ ]:
def print_gpu_usage():
    """Affiche l'utilisation du GPU"""
    if torch.cuda.is_available():
        print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
        print(f"💾 Mémoire utilisée: {torch.cuda.memory_allocated(0)/1024**3:.2f} GB")
        print(f"💾 Mémoire réservée: {torch.cuda.memory_reserved(0)/1024**3:.2f} GB")
        print(f"🔥 Utilisation: {torch.cuda.utilization(0)}%")

# Appeler avant l'entraînement
print_gpu_usage()

In [ ]:
# Visualisation des Courbes d'Entraînement
def plot_training_history(history):
    """Visualise les courbes d'entraînement"""
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    epochs = range(1, len(history['train_loss']) + 1)
    
    # Loss
    axes[0, 0].plot(epochs, history['train_loss'], 'b-o', label='Train Loss', linewidth=2)
    axes[0, 0].plot(epochs, history['val_loss'], 'r-o', label='Val Loss', linewidth=2)
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Accuracy
    axes[0, 1].plot(epochs, history['train_acc'], 'b-o', label='Train Acc', linewidth=2)
    axes[0, 1].plot(epochs, history['val_acc'], 'r-o', label='Val Acc', linewidth=2)
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy (%)')
    axes[0, 1].set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Learning Rate
    axes[1, 0].plot(epochs, history['lr'], 'g-o', linewidth=2)
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Learning Rate')
    axes[1, 0].set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
    axes[1, 0].set_yscale('log')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Overfitting gap
    gap = [t - v for t, v in zip(history['train_acc'], history['val_acc'])]
    axes[1, 1].plot(epochs, gap, 'm-o', linewidth=2)
    axes[1, 1].axhline(y=0, color='k', linestyle='--', alpha=0.3)
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Accuracy Gap (%)')
    axes[1, 1].set_title('Train-Val Accuracy Gap (Overfitting)', fontsize=14, fontweight='bold')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(CONFIG['FIG_DIR'] / 'training_curves.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Statistiques
    print("\n📊 STATISTIQUES D'ENTRAÎNEMENT:")
    print("=" * 60)
    print(f"Meilleure val_acc: {max(history['val_acc']):.2f}% (epoch {history['val_acc'].index(max(history['val_acc']))+1})")
    print(f"Meilleure val_loss: {min(history['val_loss']):.4f} (epoch {history['val_loss'].index(min(history['val_loss']))+1})")
    print(f"Gap final train-val: {gap[-1]:.2f}%")
    print(f"Learning rate final: {history['lr'][-1]:.6f}")

# Afficher les courbes
plot_training_history(history)

In [ ]:
# Évaluation sur le Test Set
def evaluate_model(model, loader, device, classes):
    """Évalue le modèle et retourne toutes les métriques"""
    
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for images, labels in tqdm(loader, desc='Évaluation'):
            images = images.to(device)
            outputs = model(images)
            probs = torch.softmax(outputs, dim=1)
            _, predicted = outputs.max(1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.numpy())
            all_probs.extend(probs.cpu().numpy())
    
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)
    
    # Métriques
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    
    # Matrice de confusion
    cm = confusion_matrix(all_labels, all_preds)
    
    # Rapport de classification
    class_report = classification_report(all_labels, all_preds, target_names=classes, output_dict=True)
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'confusion_matrix': cm,
        'classification_report': class_report,
        'predictions': all_preds,
        'labels': all_labels,
        'probabilities': all_probs
    }

# Charger le meilleur modèle pour évaluation
print("\n🔍 CHARGEMENT DU MEILLEUR MODÈLE POUR ÉVALUATION...")
checkpoint = torch.load(best_model_path)
model.load_state_dict(checkpoint['model_state_dict'])
print(f"✅ Modèle chargé (epoch {checkpoint['epoch']}, val_loss: {checkpoint['val_loss']:.4f})")

# Évaluation sur test set
print("\n🧪 ÉVALUATION SUR LE TEST SET")
print("=" * 60)

test_metrics = evaluate_model(model, test_loader, device, CONFIG['CLASSES'])

# Affichage des résultats
print(f"\n📊 RÉSULTATS SUR LE TEST SET:")
print("=" * 60)
print(f"Accuracy:  {test_metrics['accuracy']:.4f}")
print(f"Precision: {test_metrics['precision']:.4f}")
print(f"Recall:    {test_metrics['recall']:.4f}")
print(f"F1-Score:  {test_metrics['f1_score']:.4f}")

In [ ]:
# Matrice de Confusion
plt.figure(figsize=(10, 8))
sns.heatmap(test_metrics['confusion_matrix'], 
            annot=True, fmt='d', cmap='Blues',
            xticklabels=CONFIG['CLASSES'], 
            yticklabels=CONFIG['CLASSES'])
plt.title('Matrice de Confusion - Test Set', fontsize=16, fontweight='bold')
plt.xlabel('Prédictions')
plt.ylabel('Vraies étiquettes')
plt.tight_layout()
plt.savefig(CONFIG['FIG_DIR'] / 'confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# Rapport de classification détaillé
print("\n📋 RAPPORT DE CLASSIFICATION DÉTAILLÉ:")
print("=" * 60)
print(classification_report(test_metrics['labels'], test_metrics['predictions'], 
                          target_names=CONFIG['CLASSES']))

# Sauvegarde des métriques
metrics_file = CONFIG['LOG_DIR'] / f"{CONFIG['MODEL_NAME']}_test_metrics.json"
with open(metrics_file, 'w') as f:
    json.dump({k: (float(v) if isinstance(v, (np.floating, float)) else v.tolist() if isinstance(v, np.ndarray) else v) 
              for k, v in test_metrics.items() if k not in ['predictions', 'labels', 'probabilities']}, f, indent=2)

print(f"\n💾 Métriques sauvegardées: {metrics_file}")

In [ ]:
# Visualisation des Prédictions
def visualize_predictions(model, loader, device, classes, num_samples=12):
    """Visualise des exemples de prédictions"""
    
    model.eval()
    images_list = []
    preds_list = []
    labels_list = []
    probs_list = []
    
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)
            probs = torch.softmax(outputs, dim=1)
            _, preds = outputs.max(1)
            
            # Stocker pour visualisation
            images_list.extend(images.cpu().numpy())
            preds_list.extend(preds.cpu().numpy())
            labels_list.extend(labels.numpy())
            probs_list.extend(probs.cpu().numpy())
            
            if len(images_list) >= num_samples:
                break
    
    # Sélectionner un sous-ensemble
    indices = np.random.choice(len(images_list), min(num_samples, len(images_list)), replace=False)
    
    # Créer la figure
    fig, axes = plt.subplots(3, 4, figsize=(16, 12))
    axes = axes.flatten()
    
    for idx, ax_idx in enumerate(indices):
        image = images_list[ax_idx].transpose(1, 2, 0)
        image = (image * [0.229, 0.224, 0.225]) + [0.485, 0.456, 0.406]  # Dénormaliser
        image = np.clip(image, 0, 1)
        
        true_label = labels_list[ax_idx]
        pred_label = preds_list[ax_idx]
        prob = probs_list[ax_idx][pred_label]
        
        ax = axes[idx]
        ax.imshow(image, cmap='gray')
        ax.set_title(f'True: {classes[true_label]}\nPred: {classes[pred_label]}\nConf: {prob:.3f}', 
                    fontsize=10)
        ax.axis('off')
        
        # Colorer en vert si correct, rouge si incorrect
        color = 'green' if true_label == pred_label else 'red'
        for spine in ax.spines.values():
            spine.set_edgecolor(color)
            spine.set_linewidth(3)
    
    # Cacher les axes vides
    for idx in range(len(indices), len(axes)):
        axes[idx].axis('off')
    
    plt.suptitle('Exemples de Prédictions sur le Test Set\n(Vert=Correct, Rouge=Incorrect)', 
                fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig(CONFIG['FIG_DIR'] / 'prediction_examples.png', dpi=300, bbox_inches='tight')
    plt.show()

# Visualiser quelques prédictions
print("\n🎨 VISUALISATION DES PRÉDICTIONS...")
visualize_predictions(model, test_loader, device, CONFIG['CLASSES'])

In [ ]:
# Analyse par Classe
def analyze_class_performance(test_metrics, classes):
    """Analyse détaillée des performances par classe"""
    
    print("\n📈 ANALYSE DES PERFORMANCES PAR CLASSE")
    print("=" * 60)
    
    # Extraire les métriques par classe
    class_report = test_metrics['classification_report']
    cm = test_metrics['confusion_matrix']
    
    # Créer un DataFrame pour l'analyse
    class_stats = []
    for i, class_name in enumerate(classes):
        stats = class_report[class_name]
        class_stats.append({
            'Classe': class_name,
            'Précision': f"{stats['precision']:.3f}",
            'Rappel': f"{stats['recall']:.3f}",
            'F1-Score': f"{stats['f1-score']:.3f}",
            'Support': stats['support']
        })
    
    # Afficher le tableau
    df_class_stats = pd.DataFrame(class_stats)
    print(df_class_stats.to_string(index=False))
    
    # Visualisation des métriques par classe
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Précision par classe
    precision_data = [class_report[cls]['precision'] for cls in classes]
    axes[0].bar(classes, precision_data, color='skyblue', edgecolor='black')
    axes[0].set_title('Précision par Classe', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('Précision')
    axes[0].tick_params(axis='x', rotation=45)
    
    # Rappel par classe
    recall_data = [class_report[cls]['recall'] for cls in classes]
    axes[1].bar(classes, recall_data, color='lightgreen', edgecolor='black')
    axes[1].set_title('Rappel par Classe', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('Rappel')
    axes[1].tick_params(axis='x', rotation=45)
    
    # F1-Score par classe
    f1_data = [class_report[cls]['f1-score'] for cls in classes]
    axes[2].bar(classes, f1_data, color='lightcoral', edgecolor='black')
    axes[2].set_title('F1-Score par Classe', fontsize=14, fontweight='bold')
    axes[2].set_ylabel('F1-Score')
    axes[2].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig(CONFIG['FIG_DIR'] / 'class_performance.png', dbox_inches='tight')
    plt.show()
    
    return df_class_stats

# Exécuter l'analyse
class_performance = analyze_class_performance(test_metrics, CONFIG['CLASSES'])

In [ ]:
# Validation sur l'Ensemble de Validation
print("\n🔍 VALIDATION SUR L'ENSEMBLE DE VALIDATION")
print("=" * 60)

# Évaluation sur validation set
val_metrics = evaluate_model(model, val_loader, device, CONFIG['CLASSES'])

print(f"\n📊 RÉSULTATS SUR VALIDATION SET:")
print("=" * 60)
print(f"Accuracy:  {val_metrics['accuracy']:.4f}")
print(f"Precision: {val_metrics['precision']:.4f}")
print(f"Recall:    {val_metrics['recall']:.4f}")
print(f"F1-Score:  {val_metrics['f1_score']:.4f}")

# Comparaison Test vs Validation
print("\n📈 COMPARAISON TEST vs VALIDATION")
print("=" * 60)
comparison_data = {
    'Dataset': ['Test', 'Validation'],
    'Accuracy': [test_metrics['accuracy'], val_metrics['accuracy']],
    'Precision': [test_metrics['precision'], val_metrics['precision']],
    'Recall': [test_metrics['recall'], val_metrics['recall']],
    'F1-Score': [test_metrics['f1_score'], val_metrics['f1_score']]
}

df_comparison = pd.DataFrame(comparison_data)
print(df_comparison.to_string(index=False))

# Visualisation de la comparaison
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(comparison_data['Dataset']))
width = 0.2

metrics_to_plot = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
colors = ['skyblue', 'lightgreen', 'lightcoral', 'gold']

for i, metric in enumerate(metrics_to_plot):
    ax.bar(x + i*width, comparison_data[metric], width, label=metric, color=colors[i], edgecolor='black')

ax.set_xlabel('Dataset')
ax.set_ylabel('Score')
ax.set_title('Comparaison des Performances: Test vs Validation', fontsize=14, fontweight='bold')
ax.set_xticks(x + 1.5*width)
ax.set_xticklabels(comparison_data['Dataset'])
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(CONFIG['FIG_DIR'] / 'test_val_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Résumé Final et Export
def create_training_summary(history, test_metrics, val_metrics, config, training_time):
    """Crée un résumé complet de l'entraînement"""
    
    summary = {
        'model_name': config['MODEL_NAME'],
        'training_time_minutes': training_time / 60,
        'total_epochs_trained': len(history['train_loss']),
        'best_epoch': history['val_loss'].index(min(history['val_loss'])) + 1,
        'best_val_loss': min(history['val_loss']),
        'best_val_acc': max(history['val_acc']),
        'final_train_acc': history['train_acc'][-1],
        'final_val_acc': history['val_acc'][-1],
        'test_metrics': {
            'accuracy': float(test_metrics['accuracy']),
            'precision': float(test_metrics['precision']),
            'recall': float(test_metrics['recall']),
            'f1_score': float(test_metrics['f1_score'])
        },
        'val_metrics': {
            'accuracy': float(val_metrics['accuracy']),
            'precision': float(val_metrics['precision']),
            'recall': float(val_metrics['recall']),
            'f1_score': float(val_metrics['f1_score'])
        },
        'overfitting_gap': float(history['train_acc'][-1] - history['val_acc'][-1]),
        'config': {k: v for k, v in config.items() if k not in ['SAVE_DIR', 'LOG_DIR', 'FIG_DIR', 'DATA_PATH']},
        'timestamp': time.strftime('%Y-%m-%d %H:%M:%S')
    }
    
    return summary

# Créer le résumé
summary = create_training_summary(history, test_metrics, val_metrics, CONFIG, total_time)

# Sauvegarder le résumé
summary_file = CONFIG['LOG_DIR'] / f"{CONFIG['MODEL_NAME']}_summary.json"
with open(summary_file, 'w') as f:
    json.dump(summary, f, indent=2, default=str)

print(f"\n📄 RÉSUMÉ COMPLET SAUVEGARDÉ: {summary_file}")

# Affichage final
print("\n" + "=" * 70)
print("🎉 ENTRAÎNEMENT ET ÉVALUATION TERMINÉS AVEC SUCCÈS!")
print("=" * 70)
print(f"📊 Performance finale:")
print(f"   ✅ Test Accuracy:  {test_metrics['accuracy']:.4f}")
print(f"   ✅ Test F1-Score:  {test_metrics['f1_score']:.4f}")
print(f"   ✅ Val Accuracy:   {val_metrics['accuracy']:.4f}")
print(f"   ✅ Val F1-Score:   {val_metrics['f1_score']:.4f}")
print(f"⏱️  Temps total: {total_time/60:.2f} minutes")
print(f"📁 Modèle sauvegardé: {best_model_path}")
print(f"📈 Visualisations: {CONFIG['FIG_DIR']}")
print("=" * 70)